# Analysis and Plotting of Earthquake Catalogue Location Quality Scores 

This code is designed to accompany the manuscript Warren-Smith et al., submitted to NZJGG June 2024

## 1. Imports

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import os

from shapely.geometry import Point, Polygon

from obspy import UTCDateTime, read_events, Catalog
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.client import FDSNRequestTooLargeException

from helpers.nz_polygons import KNOWN_REGIONS, extract_polygon
from helpers.catalog_functions import (
    quality_measures_check, assign_variables, min_spick_dist, 
    binary_counts)
from helpers.plotting import (
    plot_catalog, plot_azimuthal_map, plot_depth_scatter,
    plot_quality_criteria_scores, plot_quality_score_map,
    plot_quality_score_bar)

# 2. Get a catalog of interest
We can select a more precise region in the next cell, so just get a catalog that at least encompasses your region of interest.
Here we use a catalog from an FDSN client, but you could also read one from disk, or get one from another supplier.

In [ ]:
# Define client used to download catalog
client = Client("GEONET")

# Define start and end dates and any constraint on depth
starttime = UTCDateTime(2018, 1, 1) 
endtime = UTCDateTime(2023, 1, 1) 
maxdepth = 50

# Define an initial quadrilateral of interest if downloading from a Client, 
# which can then be filtered by a polygon region later.
# This is currently set up for the 'West Coast' region - see below for polygon
maxlatitude = -41.25
minlatitude = -45
minlongitude = 167.5
maxlongitude = 173.5

bounds = f"{maxlatitude}-{minlatitude}_{maxlongitude}-{minlongitude}"

In [ ]:
# Define output/input location for catalogue
cat_out = f"temp_files/{bounds}_{starttime.strftime('%Y')}_{endtime.strftime('%Y')}_depth{maxdepth}.xml"

# We will store temporary files in the temp_files directory.
if not os.path.isdir("temp_files"):
    os.makedirs("temp_files")

#if haven't alreasy generated catalogue make it from client, or read in regional catalog if exists already
if not os.path.exists(cat_out):
    print("No existing catalogue for selection: downloading from client")
    fdsn_kwargs = dict(
        maxlatitude=maxlatitude, minlatitude=minlatitude,
        maxlongitude=maxlongitude, minlongitude=minlongitude, 
        maxdepth=maxdepth)
    try:
        cat = client.get_events(
            starttime=starttime, endtime=endtime, **fdsn_kwargs)
    except FDSNRequestTooLargeException:
        print("Request too large, splitting into chunks")
        chunk_size = 365 * 86400
        _starttime, _endtime = starttime, starttime + chunk_size
        cat = Catalog()
        while _endtime < endtime:
            print(f"Getting catalog between {_starttime} and {_endtime}")
            cat += client.get_events(
                _starttime, _endtime, **fdsn_kwargs)
            _starttime += chunk_size
            _endtime += chunk_size
        # Get the last bit
        print(f"Getting catalog between {_starttime} and {endtime}")
        cat += client.get_events(
            _starttime, endtime, **fdsn_kwargs)
    print(f"Read in {len(cat)} events into cat")
    #cat.write(cat_out, format='QUAKEML')
else:
    print("Existing catalogue found for selection: reading in from file")
    cat = read_events(cat_out)
    print(f"Read in {len(cat)} events into cat")


Let's check to see what the average depth of events in the catalogue is:

In [ ]:
# Output median and mean depths of events from catalogue:
depths = np.array([(ev.preferred_origin() or ev.origins[-1]).depth for ev in cat])
depths /= 1000  # Convert to km

print(f"Median event depth in catalogue = {np.median(depths):.2f} km")
print(f"Mean event depth in catalogue = {np.mean(depths):.2f} km")

Here we read in the station inventory to get the information needed to make calculations of the quality criteria

In [ ]:
inv_kwargs = dict(
    network="NZ", station="*", location="*",
    starttime=starttime, endtime=endtime, level='channel')
bbinv = client.get_stations(channel="HH?", **inv_kwargs)
spinv = client.get_stations(channel="EH?", **inv_kwargs)
sminv = client.get_stations(channel="BN?", **inv_kwargs)
sminv += client.get_stations(channel="HN?", **inv_kwargs)

bblons = [s.longitude for n in bbinv for s in n]
bblats = [s.latitude for n in bbinv for s in n]

splons = [s.longitude for n in spinv for s in n]
splats = [s.latitude for n in spinv for s in n]

inv = bbinv + spinv + sminv

Due to variability in cataloging methods, not all events have the quality measures that we need. In this cell we check all the events have quality metrics, and compute the metrics we need (azimuthal gap and minimum station distance) for any events without those metrics.

In [ ]:
print('Checking and calculating quality measures for in catalog')
cat = quality_measures_check(catalog=cat, inventory=inv)
print('Checked')

## 3. Define Polygons and Extract Events

We define specific regions known in nz_polygons, but you could define any shapely Polygon that suits you, or skip this step if you have already restricted your catalog how you want it.

In [ ]:
print(f"Known regions to select from: \n{KNOWN_REGIONS.keys()}")
region = 'West_Coast'

In [ ]:
#Define polygon based on pre-defined region in NZ_Polygons.py

poly = extract_polygon(region=region)
print(poly)

print(f"Extracting events within defined polygon for {region} region")
cat_poly = Catalog()
for ev in cat:
    x = Point(ev.preferred_origin().longitude, ev.preferred_origin().latitude)
    if poly.contains(x):
        cat_poly.append(ev)
    else:
        continue
print(f"Found {len(cat_poly)} events within polygon")



In [ ]:
# write out polygon catalogue if it doesn't already exist
if not os.path.exists(cat_poly_out):
        cat_poly.write(cat_poly_out, format='QUAKEML')
else:
    print(f"Reading from {cat_poly_out}")
    cat_poly = read_events(cat_poly_out)

## 4. Extract Key Variables from Events and Plot Catalogue

Let's extract the key information about origin locations and network coverage from the catalogue

In [ ]:
print('Assigning variables')
max_gap, depths, min_dist, mags, lats, lons, fixed = assign_variables(cat_poly=cat_poly, inventory=inv)
print('Variables assigned')

In [ ]:
#Plot Catalogue
fig = plot_catalog(
    region=region, lats=lats, lons=lons, 
    mags=mags, depths=depths, poly=poly, splons=splons, 
    splats=splats, bblons=bblons, bblats=bblats)

#save figure
fig.savefig(f'plots/cat_map_{region}_{maxdepth}.eps', dpi=800, bbox_inches='tight')
fig.savefig(f'plots/cat_map_{region}_{maxdepth}.png', dpi=800, bbox_inches='tight')

## Plot Azimuthal Coverage

Make a plot of all the earthquakes, coloured by their maximum azimuthal gap (red shades: gap > 180 degrees, blue shades: gap < 180 degrees)

In [ ]:
fig = plot_azimuthal_map(lats=lats, lons=lons, max_gap=max_gap, poly=poly, region=region,
                         bblons=bblons, bblats=bblats, splons=splons, splats=splats)


fig.savefig(f'plots/azimuthal_constraints_map_{region}_{maxdepth}.eps', dpi=800, bbox_inches='tight')
fig.savefig(f'plots/azimuthal_constraints_map_{region}_{maxdepth}.png', dpi=800, bbox_inches='tight')

## Plot Min_Distance Criteria

Make a plot showing the depth of each earthquake against distance to the closest picked station. Events plotting to the lower left of the solid black line have a station within one focal depth's distance. Same for dashed black line, but within 1.4* focal depth's distance. Events plotting to the right of both lines have no pick within 1.4* focal depth's distance.

In [ ]:
# Ascertain if s-pick exists within 1.4*depth epicentral distance for each event
s_picks = min_spick_dist(cat_poly=cat_poly, inventory=inv)

# plot depth scatter plot
fig = plot_depth_scatter(min_dist=min_dist, s_picks=s_picks, depths=depths, fixed=fixed, region=region)

fig.savefig(f'plots/min_distance_constraints_{region}_{maxdepth}.eps', dpi=800, bbox_inches='tight')
fig.savefig(f'plots/min_distance_constraints_{region}_{maxdepth}.png', dpi=800, bbox_inches='tight')

## Calculate Criteria Scores and Quality Scores

Make a bar plot showing percentages of events satisfying each of the six quality criteria. Orange = pass criteria, peach = fail criteria.

In [ ]:
# Create binary counts for each quality criteria
# 1 indicates criteria satisfied, 0 indicates criteria not satisfied

min_az_bi, min_picks, ps, min_dist_bi, fixed_inv = binary_counts(max_gap=max_gap, cat_poly=cat_poly, 
                                                                 depths=depths, min_dist=min_dist, fixed=fixed)

# plot these up
fig, counts, criteria = plot_quality_criteria_scores(fixed_inv=fixed_inv, cat_poly=cat_poly, s_picks=s_picks, min_dist_bi=min_dist_bi,
                                 ps=ps, min_picks=min_picks, min_az_bi=min_az_bi, region=region)
fig.savefig(f'plots/quality_criteria_6_hist_{region}_{maxdepth}.eps', dpi=800, bbox_inches='tight')
fig.savefig(f'plots/quality_criteria_6_hist_{region}_{maxdepth}.png', dpi=800, bbox_inches='tight')


Print out these percentages to screen

In [ ]:
from IPython.display import display, Latex

# Percentages of events satisfying each criteria:
print('Percentages of events satisfying each criteria:')
for i, c in enumerate(criteria):
    display(Latex(f"{c}: {counts[i]:.1f}"))

In [ ]:
# calculate the percentage of scores and plot map and pie chart 
sums=[]
counts=[]
for i, ev in enumerate(cat_poly):
    sums.append(min_az_bi[i] + ps[i] + min_picks[i] + fixed_inv[i] + min_dist_bi[i] + s_picks[i])
quals=np.arange(0, 7, 1)
for n in quals:
    counts.append(sums.count(n))

## NOTE: if depth fixing is not used in selected catalogue, a 5-QS scoring system should be employed. 
## i.e. disregard QS6 and use QS5 as the mazimum possible score    

# plot map    
fig = plot_quality_score_map(
    quals=quals, counts=counts, lons=lons, lats=lats, region=region, poly=poly,
    cat_poly=cat_poly, sums=sums, bblons=bblons, bblats=bblats, splons=splons, splats=splats)

fig.savefig(f'plots/quality_constraints_6_map_{region}_{maxdepth}.eps', dpi=800, bbox_inches='tight')
fig.savefig(f'plots/quality_constraints_6_map_{region}_{maxdepth}.png', dpi=800, bbox_inches='tight')

In [ ]:
# Extract the percentages for each QS and also plot as bar chart

countspc, countspc_cum=[],[]
for c in counts:
    countspc.append(c/len(sums)*100)
countspc_cum=np.cumsum(countspc)

print('Final Quality Score Percentages for ' + region + ' region are:')
for i, QS in enumerate(['QS0', 'QS1', 'QS2', 'QS3', 'QS4', 'QS5', 'QS6']):
    print(QS + ': ' + str(round(countspc[i],1)))

# plot as bar chart

fig=plot_quality_score_bar(region=region, countspc_cum=countspc_cum, quals=quals)
    
fig.savefig(f'plots/quality_constraints_6_barchart_{region}_{maxdepth}.eps', dpi=800, bbox_inches='tight')
fig.savefig(f'plots/quality_constraints_6_barchart_{region}_{maxdepth}.png', dpi=800, bbox_inches='tight')

Finally, write out the catalogue with the Quality Score saved within the preferred (or last) origin's comments

In [ ]:
from obspy.core.event.base import Comment
for i, ev in enumerate(cat_poly):
    quality_score=sums[i]
    (ev.preferred_origin() or ev.origins[-1]).comments.append(Comment(text=f"Quality score: {quality_score}"))
    
cat_out = f"temp_files/{region}_{starttime.strftime('%Y')}_{endtime.strftime('%Y')}_{maxdepth}_quality_scores.xml"
cat.write(cat_out, format='QUAKEML')